In [1]:
from pyspark.sql.types import *

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("HousingPrices") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()



base = spark.read.csv("file:///home/maria/Documents/SparkSQLPractice/climate_change.csv",inferSchema=True,header=True)
print(base.head())
print(base.printSchema())
print(base.columns)

Row(Year=1983, Month=5, MEI=2.556, CO2=345.96, CH4=1638.59, N2O=303.677, CFC-11=191.324, CFC-12=350.113, TSI=1366.1024, Aerosols=0.0863, Temp=0.109)
root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- MEI: double (nullable = true)
 |-- CO2: double (nullable = true)
 |-- CH4: double (nullable = true)
 |-- N2O: double (nullable = true)
 |-- CFC-11: double (nullable = true)
 |-- CFC-12: double (nullable = true)
 |-- TSI: double (nullable = true)
 |-- Aerosols: double (nullable = true)
 |-- Temp: double (nullable = true)

None
['Year', 'Month', 'MEI', 'CO2', 'CH4', 'N2O', 'CFC-11', 'CFC-12', 'TSI', 'Aerosols', 'Temp']


In [ ]:
'''
Question ID: 10101
We are interested in how changes in these variables affect future temperatures, as well as how well these variables explain temperature changes so far. To do this, first read the dataset climate_change.csv into R. 

Then, split the data into a training set, consisting of all the observations up to and including 2006, and a testing set consisting of the remaining years (hint: use subset). A training set refers to the data that will be used to build the model (this is the data we give to the lm() function), and a testing set refers to the data we will use to test our predictive ability.

Next, build a linear regression model to predict the dependent variable Temp, using MEI, CO2, CH4, N2O, CFC.11, CFC.12, TSI, and Aerosols as independent variables (Year and Month should NOT be used in the model). Use the training set to build the model.

Enter the model R2 (the Multiple R2 value).

Attempts: 0 / 2
'''


In [5]:
from pyspark.sql.functions import col
df_train = base.where(col('Year') <= 2006) 
df_test = base.where(col('Year')>2006)
df_train.show()
df_test.show()

+----+-----+------+------+-------+-------+-------+-------+---------+--------+------+
|Year|Month|   MEI|   CO2|    CH4|    N2O| CFC-11| CFC-12|      TSI|Aerosols|  Temp|
+----+-----+------+------+-------+-------+-------+-------+---------+--------+------+
|1983|    5| 2.556|345.96|1638.59|303.677|191.324|350.113|1366.1024|  0.0863| 0.109|
|1983|    6| 2.167|345.52|1633.71|303.746|192.057|351.848|1366.1208|  0.0794| 0.118|
|1983|    7| 1.741|344.15|1633.22|303.795|192.818|353.725| 1366.285|  0.0731| 0.137|
|1983|    8|  1.13|342.25|1631.35|303.839|193.602|355.633|1366.4202|  0.0673| 0.176|
|1983|    9| 0.428|340.17| 1648.4|303.901|194.392|357.465|1366.2335|  0.0619| 0.149|
|1983|   10| 0.002| 340.3|1663.79| 303.97|195.171|359.174|1366.0589|  0.0569| 0.093|
|1983|   11|-0.176|341.53|1658.23|304.032|195.921|360.758|1366.1072|  0.0524| 0.232|
|1983|   12|-0.176|343.07|1654.31|304.082|196.609|362.174|1366.0607|  0.0486| 0.078|
|1984|    1|-0.339|344.05|1658.98| 304.13|197.219|363.359|1365.42

In [23]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
feature_cols=['MEI', 'CO2', 'CH4', 'N2O', 'CFC-11', 'CFC-12', 'TSI', 'Aerosols']

assembler = VectorAssembler(inputCols=feature_cols,outputCol="features")
for_training_df = assembler.transform(df_train)
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
                        withStd=True, withMean=True)

# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(for_training_df)

# Normalize each feature to have unit standard deviation.
scaledData = scalerModel.transform(for_training_df)

lr = LinearRegression(featuresCol = "features", labelCol='Temp', maxIter=100, regParam=0.0, elasticNetParam=0.8)
lr_model = lr.fit(scaledData)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)
print(trainingSummary.pValues)

Coefficients: [0.06420531326591102,0.006457359294170654,0.00012404189957649487,-0.0165280033609315,-0.006630488896080285,0.003808103247995261,0.09314108244845155,-1.5376132353114387]
Intercept: -124.59425896818887
RMSE: 0.090247
r2: 0.750893
[0.0, 0.00505252047244098, 0.810145546547842, 0.05466931039549405, 5.957287568936209e-05, 0.00020971989521445522, 1.0959446683500573e-09, 5.411227022023013e-12, 1.4310466145417422e-09]
